In [1]:
import numpy as np
from osgeo import gdal
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

--- Girdilerin tanımı ve Görüntünün Array Hale Getirilmesi ---

In [2]:
inpRaster = 'D:/TEZ/SentinelGoruntu/Islem_Adimleri/4_Kompozit/04_NISAN/04_Kompozit.tif'
outRaster = 'D:/TEZ/SentinelGoruntu/Siniflama_test/output.tif'
#df = pd.read_csv('')

ds = gdal.Open(inpRaster, gdal.GA_ReadOnly)

rows = ds.RasterYSize
cols = ds.RasterXSize
bands = ds.RasterCount
geo_transform = ds.GetGeoTransform()
projection = ds.GetProjectionRef()


print(rows, cols)
print(bands)

array = ds.ReadAsArray()
ds = None

#print(array)

D:\Conda\envs\RF_Yuksek\lib\site-packages\osgeo\gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


20982 10980
10


--- Görüntüyü ve Train verisini Array Yapalım ----

In [3]:
array = np.stack(array, axis=2)
array = np.reshape(array, [rows*cols,bands])

print(array[0])

test = pd.DataFrame(array, dtype='int16')
del array

[ 497  553  384 1211 2177 3254 1840 3559 3223 2221]


In [ ]:
# Training data verilerinin belirlenmesi
data = df['test verisindeki band sütun isimleri']
label = df['test verisindeki ürün idleri']

--- Model Eğitimi ----

In [ ]:
# Sınıflayıcı parametrelerinin ayarlanması
clf = RandomForestClassifier(n_estimator=50, n_jobs=-1)
clf.fit(data,label)
del data
del label

In [ ]:
# Sınıfların predict edilmesi
y_pred = clf.predict(test)
del test

In [ ]:
classification = y_pre.reshape((rows,cols))
del y_pred

--- Sınıflanmış olan Görüntünün Oluşturulması ---

In [ ]:
def createGeotiff(outRaster, data, geo_transform, projection):
    # veri ile GEOTIFF oluşturalım
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    rasterDS = driver.Create(outRaster, cols, rows, 1, gdal.GDT_Int32)
    rasterDS.SetGeoTransform(geo_transform)
    rasterDS.SetProjection(projection)
    band = rasterDS.GetRasterBand(1)
    band.WriteArray(data)
    rasterDS = None


#Sınıflanan Görüntü export edilir
createGeotiff(outRaster,classification,geo_transform,projection)